In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Activation, Dropout, GlobalAveragePooling2D

In [4]:
CLASSES = 2 # missing pulley or perfect pulley

baseModel = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = baseModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
#x = Dense(1024, activation='relu')(x)
#x = Dense(512, activation='relu')(x)
#x = Dropout(0.5)(x)
preds = Dense(CLASSES, activation='softmax')(x)

model = Model(inputs=baseModel.input, outputs=preds)
#model.summary()

In [ ]:
for i, layer in enumerate(model.layers):
    print("{}: {}".format(i, layer))

In [6]:
for layer in model.layers[:87]:
    layer.trainable = False

for layer in model.layers[87:]:
    layer.trainable = True

In [7]:
trainDataGen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split = 0.3)

trainGen = trainDataGen.flow_from_directory('/home/dongjun/djplace/samples/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            #color_mode = 'rgb',
                                            class_mode = 'categorical',
                                            subset = 'training',
                                            shuffle = True)

validGen = trainDataGen.flow_from_directory('/home/dongjun/djplace/samples/',
                                           target_size = (224, 224),
                                           batch_size = 32,
                                           class_mode = 'categorical',
                                           subset = 'validation')

Found 7469 images belonging to 2 classes.


In [9]:
FILEPATH = '/home/dongjun/djplace/samples/'
EPOCHS = 10
LEARNINGRATE = 0.001
DECAYRATE = LEARNINGRATE // EPOCHS
opt = Adam(lr = LEARNINGRATE, decay=DECAYRATE)
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit_generator(generator=trainGen,
                    steps_per_epoch = trainGen.n // trainGen.batch_size,
                    validation_data = validGen,
                    validation_steps = validGen.n // validGen.batch_size,
                    epochs = EPOCHS)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
233/233 [==============================] - 895s 4s/step - loss: 0.4031 - accuracy: 0.8388 - val_loss: 1.3504 - val_accuracy: 0.5734
Epoch 2/10
233/233 [==============================] - 990s 4s/step - loss: 0.2241 - accuracy: 0.9061 - val_loss: 1.8681 - val_accuracy: 0.5469
Epoch 3/10
233/233 [==============================] - 1003s 4s/step - loss: 0.1351 - accuracy: 0.9433 - val_loss: 2.1968 - val_accuracy: 0.5809
Epoch 4/10
233/233 [==============================] - 1006s 4s/step - loss: 0.1549 - accuracy: 0.9332 - val_loss: 3.6786 - val_accuracy: 0.5163
Epoch 5/10
233/233 [==============================] - 1008s 4s/step - loss: 0.1283 - accuracy: 0.9457 - val_loss: 1.2542 - val_accuracy: 0.6488
Epoch 6/10
233/233 [==============================] - 973s 4s/step - loss: 0.1257 - accuracy: 0.9466 - val_loss: 1.3152 - val_accuracy: 0.6156
Epoch 7/10
233/233 [==============================] - 974s 4s/step - loss: 0.1317 - accurac